# Shallow Water Equations in 2D with Arakawa C grid

In [16]:
function swe_arakawa(Nx::Integer, Ny::Integer, dx::T, dy::T, dt::T, time_steps::Integer, h0::Function, f::T, gamma::T=T(0.1), g::T=T(9.81), H::T=T(4000.0)) where {T<:AbstractFloat}
    u = zeros(Nx, Ny, time_steps)
    v = zeros(Nx, Ny, time_steps)
    h = zeros(Nx, Ny, time_steps)

    x = collect(range(start=T(0.0), step=dx, length=Nx))
    y = collect(range(start=T(0.0), step=dy, length=Ny))

    i = collect(1:Nx)
    j = collect(1:Ny)
    ip1 = circshift(i, -1)
    im1 = circshift(i, 1)
    jp1 = circshift(j, -1)
    jm1 = circshift(j, 1)

    # apply initial conditions for the height
    h[i, j, 1] = @. h0(x, y')

    k = 1 # euler step
    # compute rhs for the method of lines
    dudt = -g / dx * (h[ip1, j, k] - h[i, j, k]) + 0.25 * f * (v[i, j, k] + v[ip1, j, k] + v[ip1, jm1, k] + v[i, jm1, k])
    dvdt = -g / dy * (h[i, jp1, k] - h[i, j, k]) + 0.25 * f * (u[i, j, k] + u[i, jp1, k] + u[im1, jp1, k] + u[im1, j, k])
    dhdt = - H * ((u[i, j, k] - u[im1, j, k]) / dx + (v[i, j, k] - v[i, jm1, k]) / dy)

    # perform euler step for time integration
    u[i, j, k+1] = u[i, j, k] + dt * dudt
    v[i, j, k+1] = v[i, j, k] + dt * dvdt
    h[i, j, k+1] = h[i, j, k] + dt * dhdt

    # leap frog
    for k = 2:time_steps-1
        dudt = -g / dx * (h[ip1, j, k] - h[i, j, k]) + 0.25 * f * (v[i, j, k] + v[ip1, j, k] + v[ip1, jm1, k] + v[i, jm1, k])
        dvdt = -g / dy * (h[i, jp1, k] - h[i, j, k]) + 0.25 * f * (u[i, j, k] + u[i, jp1, k] + u[im1, jp1, k] + u[im1, j, k])
        dhdt = - H * ((u[i, j, k] - u[im1, j, k]) / dx + (v[i, j, k] - v[i, jm1, k]) / dy)

        u[i, j, k+1] = u[i, j, k-1] + 2 * dt * dudt
        v[i, j, k+1] = v[i, j, k-1] + 2 * dt * dvdt
        h[i, j, k+1] = h[i, j, k-1] + 2 * dt * dhdt

        u[i, j, k] = u[i, j, k] + gamma * (u[i, j, k-1] - 2 * u[i, j, k] + u[i, j, k+1])
        v[i, j, k] = v[i, j, k] + gamma * (v[i, j, k-1] - 2 * v[i, j, k] + v[i, j, k+1])
        h[i, j, k] = h[i, j, k] + gamma * (h[i, j, k-1] - 2 * h[i, j, k] + h[i, j, k+1])
    end

    return u, v, h
end

swe_arakawa (generic function with 4 methods)

In [29]:
Nx = 101
Ny = 101
dx = 5e5
dy = 5e5
dt = 10.0
h0(x,y) = (45 * dx <= x <= 55 * dx &&  45 * dy <= y <= 55 * dy ) ? 1 : 0
f = 0.0
time_steps = 1200

u, v, h = swe_arakawa(Nx, Ny, dx, dy,dt, time_steps, h0, f)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; … ;;; -3.2886760689159785e-182 -2.337059629354147e-179 … -2.337059629354147e-179 -3.2886760689159785e-182; -2.2882009640949663e-179 -1.607343892391935e-176 … -1.607343892391935e-176 -2.2882009640949663e-179; … ; 3.2886760689159785e-182 2.337059629354147e-179 … 2.337059629354147e-179 3.2886760689159785e-182; 0.0 0.0 … 0.0 0.0;;; -3.818156932184964e-182 -2.708726369066679e-179 … -2.708726369066679e-179 -3.818156932184964e-182; -2.6521036819914266e-179 -1.859807869776692e-176 … -1.859807869776692e-176 -2.6521036819914266e-179; … ; 3.818156932184964e-182 2.708726369066679e-179 … 2.708726369066679e-179 3.818156932184964e-182; 0.0 0.0 … 0.0 0.0;;; -4.421608024088539e-182 -3.1316887514930045e-179 … -3.1316887514930045e-179 -4.421608024088539e-182; -

In [18]:
h

11×11×120 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0       0.0       0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0       0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0       0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0       0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       7.848e-5  0.0       0.0  0.0  

In [23]:
using Plots

In [31]:
"""
create_quiver_gif(u, v, h; k=8, m=10, dx=1.0, dy=1.0, filename="anim.gif",
                   cmap=:viridis, quiver_scale=1.0, fps=10)

Create a GIF visualizing the vector field (u,v) as a quiver and the scalar field h as a colormapped
background. The arrays u,v,h are expected to be (Nx, Ny, Nt).

Arguments:
- u, v, h : Float arrays of shape (Nx, Ny, Nt)
Keyword arguments:
- k         : integer ≤ 9, choose k x k arrows (default 8). Must be >= 1.
- m         : integer ≥ 1, output a frame every m time steps (default 10)
- dx, dy    : physical spacing in x and y (default 1.0)
- filename  : output GIF filename (default "anim.gif")
- cmap      : colormap symbol for the background (default :viridis)
- quiver_scale : scalar multiplied into arrow components (default 1.0)
- fps       : frames per second for the GIF (default 10)

Returns nothing; writes filename to disk.
"""
function create_quiver_gif(u::AbstractArray, v::AbstractArray, h::AbstractArray;
                           k::Integer=8, m::Integer=10,
                           dx::Real=5e5, dy::Real=5e5,
                           filename::String="anim.gif",
                           cmap=:viridis, quiver_scale::Real=1.0,
                           fps::Integer=10)

    # basic validation
    Nx, Ny, Nt = size(u)
    @assert size(v) == (Nx, Ny, Nt) "v must have same shape as u"
    @assert size(h) == (Nx, Ny, Nt) "h must have same shape as u"
    @assert 1 ≤ k ≤ min(Nx, Ny) "k must be between 1 and min(Nx,Ny)"
    @assert k < 10 "please choose k < 10 as requested"
    @assert m ≥ 1 "m must be ≥ 1"

    # physical coordinates
    x = collect(0:dx:(Nx-1)*dx)
    y = collect(0:dy:(Ny-1)*dy)

    # choose evenly spaced sample indices (k per axis)
    ix = round.(Int, range(1, stop=Nx, length=k))
    jy = round.(Int, range(1, stop=Ny, length=k))

    # sample coordinates (match the order of vec(...) for a k×k matrix)
    xs = x[ix]
    ys = y[jy]
    # vec(u[ix, jy]) stacks columns -> for j in 1:k, for i in 1:k
    xq = repeat(xs, outer=k)   # x varies fastest within each column set
    yq = repeat(ys, inner=k)   # each y repeated k times for its column

    # global color limits so color is consistent across frames
    hmin = minimum(h)
    hmax = maximum(h)

    anim = @animate for t in 1:m:Nt
        # background: heatmap of h at time t
        # note: transpose h[:,:,t] so axes align intuitively (x horizontal, y vertical)
        plt = heatmap(x, y, h[:, :, t]',
                      color = cmap,
                      clims = (hmin, hmax),
                      aspect_ratio = :equal,
                      xlabel = "x", ylabel = "y",
                      title = "t = $(t)",
                      framestyle = :box,
                      colorbar = true)

        # subsampled vector components (must be flattened in the same column-major order)
        u_sub = vec(u[ix, jy, t]) .* quiver_scale
        v_sub = vec(v[ix, jy, t]) .* quiver_scale

        # overlay quiver (arrow color chosen to be black for contrast)
        quiver!(plt, xq, yq, quiver = (u_sub, v_sub),
                linewidth = 1.2, arrow = :arrow, linecolor = :black, legend = false)

        plt
    end

    # write gif
    gif(anim, filename; fps = fps)
    return nothing
end

create_quiver_gif

In [32]:
create_quiver_gif(u, v, h;k=8, m=10,dx=5e5, dy=5e5, filename="anim.gif", cmap=:RdBu , quiver_scale=2.0,fps=10)

Wrote GIF to: anim.gif


[ Info: Saved animation to /home/niclas/uni_leipzig/07_Veranstaltungen_SoSe25/Meterologie/PraktikumsBericht/src/anim.gif


In [36]:
function plot_height_heatmap(height, times, filename,dx::Real=5e5, dy::Real=5e5, cmap::Symbol=:RdBu)
    hmin = minimum(h)
    hmax = maximum(h)

    Nx, Ny, Nt = size(u)
    @assert size(v) == (Nx, Ny, Nt) "v must have same shape as u"
    @assert size(h) == (Nx, Ny, Nt) "h must have same shape as u"

    # physical coordinates
    x = collect(0:dx:(Nx-1)*dx)
    y = collect(0:dy:(Ny-1)*dy)
    
    for k in times
        plt = heatmap(x, y, h[:, :, k]',
                          color = cmap,
                          clims = (hmin, hmax),
                          aspect_ratio = :equal,
                          xlabel = "x", ylabel = "y",
                          title = "h in 2D SWE at t = $(k)",
                          framestyle = :box,
                          colorbar = true)
        filename_adjusted = filename * "_$(k).png"
        savefig(plt, filename_adjusted)
    end
end

plot_height_heatmap(h, [1, 100, 200, 300], "heatmap")